In [1]:
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 6.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 15.4 MB/s eta 0:00:00


In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv
import pandas as pd
import re
from datasets import load_dataset, DatasetDict, load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_metric


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Variables for classification
train_dataset_path = '/content/drive/MyDrive/CS544/Project/webis-clickbait-22/train.jsonl'
valid_dataset_path = '/content/drive/MyDrive/CS544/Project/webis-clickbait-22/validation.jsonl'

# Load data
train_jsonObj = pd.read_json(path_or_buf=train_dataset_path, lines=True)
train_raw_data_list = train_jsonObj.values.tolist()

val_jsonObj = pd.read_json(path_or_buf=valid_dataset_path, lines=True)
valid_raw_data_list = val_jsonObj.values.tolist()

In [ ]:
train_l = train_jsonObj['tags'].tolist()
new_l = []
for l in train_l:
  new_l.append(l[0])
train_jsonObj['tags'] = new_l

val_l = val_jsonObj['tags'].tolist()
new_l = []
for l in val_l:
  new_l.append(l[0])
val_jsonObj['tags'] = new_l

train_jsonObj = train_jsonObj.loc[train_jsonObj['tags']=='multi']
val_jsonObj = val_jsonObj.loc[val_jsonObj['tags']=='multi']

train_data = train_jsonObj[['postText', 'targetParagraphs', 'spoiler']]
val_data = val_jsonObj[['postText', 'targetParagraphs', 'spoiler']]

In [ ]:
train_data

,postText,targetParagraphs,spoiler
3,[Passion is overrated — 7 work habits you need...,"[It’s common wisdom. Near gospel really, and n...",[Purpose connects us to something bigger and i...
9,[Hot Sauce Taste Test: Find out which we named...,[We tried 29 different kinds of hot sauce to f...,"[Sriracha Hot Chili Sauce, Frank's RedHot Orig..."
20,[Human remains found in the search for missing...,[Authorities have uncovered human remains duri...,"[Southampton County, Anjelica 'AJ' Hadsell]"
43,[Six lessons from the godfather of California ...,[When asked to describe the current state of C...,"[Daniel Patterson, 1) Eat your veggies., 2) En..."
44,[The Fastest Growing Economy in the World! Num...,"[By Jeff Russell, B. Comm., MA Published : Sep...","[1. India, 2. Bangladesh, 3. China, 4. Indones..."
...,...,...,...
3188,[Medical marijuana bills advance in these Sout...,[Georgia and Kentucky advanced bills to legali...,"[Georgia, Kentucky]"
3190,[This Guy Cheated On 'The Price Is Right' And ...,[You may have watched The Price is Right when ...,"[Terry Kniess, All of the prices in the show h..."
3191,[7 small changes to try this week],"[Small change, big difference., 1. If you have...","[Relax with rain sounds., Put pressure on peri..."
3192,"[11 of the weirdest foods you should try, acco...","[Life can get a bit vanilla sometimes, can't i...",[Cream cheese with a bit of strawberry jam on ...


In [ ]:
prompt_postText_1 = train_data.loc[43]['postText'][0]
prompt_targetParagraphs_1 = ' '.join(train_data.loc[43]['targetParagraphs'])
prompt_spoiler_1 = '\n'.join(train_data.loc[43]['spoiler'])

prompt_postText_2 = train_data.loc[20]['postText'][0]
prompt_targetParagraphs_2 = ' '.join(train_data.loc[20]['targetParagraphs'])
prompt_spoiler_2 = '\n'.join(train_data.loc[20]['spoiler'])

print(prompt_spoiler_1)
print(prompt_spoiler_2)

Daniel Patterson
1) Eat your veggies.
2) Enjoy said veggies a few weeks after their season starts.
3) Ingredients dictate everything.
4) Don't serve complex foods to your 4-year-old.
5) You can succeed without a mentor.
Southampton County
Anjelica 'AJ' Hadsell


In [ ]:
val_data

,postText,targetParagraphs,spoiler
1,[Why Arizona Sheriff Joe Arpaio’s fate could h...,[© REUTERS/Laura Segall Maricopa County Sherif...,"[""intentionally"", could transform a court case..."
3,"[""Harry Potter"" alums reunite for new movie]",[The mythology of punk music's evolution can b...,"[Alan Rickman & Rupert Grint, CBGB]"
10,[How 1 model prepares for lingerie shoots],[The life of a lingerie model is quite a forei...,"[Elettra Wiedemann, extra strength work, so we..."
13,[The question former O.J. Simpson prosecutor h...,[As the lead prosecutor in the O.J. Simpson mu...,"[Marcia Clark, Does she think Simpson really d..."
14,[Hollywood legend's amazing home is on the mar...,[Hollywood legend Mitzi Gaynor is reportedly s...,"[Mitzi Gaynor, Beverly Hills, California]"
...,...,...,...
785,[6 transcendental festivals out in the middle ...,"[Some festivals are about music, some are abou...","[BOOM – Portugal, Meadows in the Mountains – B..."
788,[Things in the Olsen twins' lives that make no...,[It kind of goes without saying that any hope ...,[earned much of their fortune making unintenti...
789,[13 Pokémon The Designers Just Gave Up On],"[With 721 to catch, Pokémon fans are bound to ...","[Klefki, Rotom, Luvdisk, Unknown (Unown), Vani..."
790,[9 things Elizabeth Taylor taught us about the...,"[Sure, she was married eight times, but there’...","[I don’t pretend to be an ordinary housewife, ..."


In [ ]:
load_dotenv()
openai_key = ''


# API configuration
openai.api_key = openai_key

# for LangChain
os.environ["OPENAI_API_KEY"] = openai_key
# os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [ ]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=32,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, prompt):
    """ GET completion from openai api"""

    response = openai.Completion.create(
        engine = params['model'],
        prompt = prompt,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

def get_chat_completion(prompt):

    response = openai.ChatCompletion.create(
        model = params['model'],
        messages = [{"role": "user", "content": prompt}],
        max_tokens = 128,
    )
    return response

In [ ]:
# I will give you a clickbait text and the corresponding linked large body of text. You have to summarize the given large body of text as an answer to the clickbait text. You have to summarize the text in the form of a list of points. I am giving you 2 examples. You must answer the next one in the same format.
#Given a clickbait text and the corresponding linked large body of text, summarize the given large body of text as an answer to the clickbait text. *Summarize the text in the form of a list of points*. ANSWER should be phrases separated by a comma and enclosed between [].

# You are given a clickbait text and the corresponding linked large body of text. Respond with a short list of phrases *extracted from the linked text* separated by "\n". 
# Your phrases must BRIEFLY summarize the given large body of text as a sarcastic answer to the clickbait text. Follow the one example given below and answer the next.

answer_postText = val_data.loc[470]['postText'][0]
answer_targetParagraphs = ' '.join(val_data.loc[470]['targetParagraphs'])
answer_spoiler = '\n'.join(val_data.loc[470]['spoiler'])

prompt = """
You are given a clickbait text and the corresponding linked large body of text. Extract the most important phrases from the linked text that can sarcastically answer the clickbait text.
*Your answer must strictly be a list with more than one phrases. The phrases should strictly be brief (one word or few words).* Follow the two examples given below and answer the next.

CLICKBAIT TEXT: {}
LINKED TEXT: {}
ANSWER: {}

CLICKBAIT TEXT: {}
LINKED TEXT: {}
ANSWER:
""".format(prompt_postText_1, prompt_targetParagraphs_1, prompt_spoiler_1, answer_postText, answer_targetParagraphs)

print(answer_spoiler)

# CLICKBAIT TEXT: "Why Humans Will Never Live Off Sunlight"
# LINKED TEXT: "C’mon, you know you want to be this guy. Image: The American Chemical Society", "Imagine if we could be like plants, lying outside all day soaking up sweet, sweet energy from the sun. Doesn’t sound like a bad life, does it? So why aren’t the world’s best minds figuring out how to hack photosynthesis into humans?", "It turns out there are a lot of reasons, which are spelled out in a new video by the American Chemical Society. First, there are aesthetics to consider: your cells would have to be packed with light-harvesting pigments called chlorophyll, meaning you’d be bright green, or maybe a custom-tailored shade of red or blue. (Although I’m pretty sure you wouldn’t have ivy sprouting out of your forehead.) But far more problematic than your skin’s unusual hue is the energetics of photosynthesis.", "Eating sunlight isn’t all that efficient—plants are only able to put a few percent of the energy packed away in the sun’s rays to work. For your average tree, that translates to about 200 calories a day. The reason our leafy green friends can get by on such a lean diet is that they live a very sedentary lifestyle, so unless you really want to lay in the sun all day and also be braindead, you’d need to supplement your solar meals with a regular intake of cheeseburgers. (Also, your body would still require protein in addition to the sugar you’re producing from photosynthesis, so again, cheeseburgers.)", "Finally, plants maximize the amount of energy they can capture by growing their leaves very thin. Your sausage fingers and burrito arms are ill suited for the task, so you’d probably want to scrap them for bionic sail appendages instead. Is this sounding fun or what!", "Insurmountable obstacles aside, the idea of hacking photosynthesis into humans has gotten real scientific consideration, and there are animals out there that really do photosynthesize, in a way. So, even though our descendants will never live entirely off the sun, who knows? Maybe distance runners of the future will be able to charge up with a little solar snack now and again.", "[American Chemical Society]"
# ANSWER: "your cells would have to be packed with light-harvesting pigments called chlorophyll", "Eating sunlight isn’t all that efficient", "plants maximize the amount of energy they can capture by growing their leaves very thin. Your sausage fingers and burrito arms are ill suited for the task".


# CLICKBAIT TEXT: "Six lessons from the godfather of California cuisine"
# LINKED TEXT: "When asked to describe the current state of California cuisine, chef Daniel Patterson replies with a simple, \"I don't think I'm qualified to answer that.\"", "Modest words from someone who has been lauded as \"a leading force in California cuisine\" and \"the man single-handedly reinventing California cuisine,\" and whose food has been revered as \"a signature cerebral brand of California cuisine.\"", "Whether he chooses to embrace these accolades or not, Patterson's cooking undoubtedly embodies the very best of California cooking, fusing fresh and seasonal local treasures with innovative preparations and presentations. The proprietor of San Francisco's double-Michelin-starred Coi shows the figs-on-a-plate crowd who's boss, transforming ingredients into the very best versions of themselves: asparagus blanched in asparagus water, strawberries coated in a strawberry glaze.", "And his influence continues to expand throughout the Bay Area and the world. Alongside the enduring popularity of Coi and his handful of restaurants in downtown Oakland, Patterson's first-ever cookbook, the aptly titled \"Coi: Stories and Recipes,\" comes out this month (he's also an avid food writer, and the book features original essays that accompany dozens of recipes). The newest addition to Patterson's ever-expanding empire will be the \"very casual\" Alta CA, slated to open in San Francisco's burgeoning Mid-Market neighborhood later this year.", "The Huffington Post recently caught up with Patterson to see if we could glean any morsels of wisdom from the self-taught chef and father of two. Herewith, a few lessons from an icon most certainly qualified to discuss the state of California cuisine and beyond.", "Patterson, right, preparing food in Madrid.", "1) Eat your veggies. Coi's evolving tasting menu is often vegetable-centric, a testament to the flavor complexity of plants. \"It's how we like to cook; it's how I like to eat,\" Patterson says, adding that Northern California is known for its produce. \"We have extraordinary vegetables here ... it's more challenging and ultimately rewarding for me to work with products that have a much greater range of flavor.\"", "2) Enjoy said veggies a few weeks after their season starts. \"We won't take things at the beginning of the season,\" Patterson, who sources his menu from a wide range of \"the very best\" foragers and producers in the Bay Area, explains. \"We wait until they are at peak flavor.\"", "3) Ingredients dictate everything. The dishes on Patterson's menus take shape only after he and his team know exactly which ingredients they'll be using. \"We mostly focus on what tastes good,\" he says. \"We'll let the ingredients talk a little bit, figure out where they want to go at any given time.\"", "4) Don't serve complex foods to your 4-year-old. When asked if he feeds his son dishes from the Coi menu, Patterson responds with, \"Hell no!\" At home, he says, he focuses on simple meals. \"We have a slow cooker,\" he says. \"I'll roast a chicken, throw the bones in with some water. Add some vegetables, take the bones out, put in some beans, turn that into dinner with some greens.\"", "5) You can succeed without a mentor. Patterson never attended culinary school or worked under a renowned chef, instead opting to open his first restaurant in Sonoma at the age of 25. \"I was able to start cooking and figure out what my style was without any preconceived ideas,\" he says. \"At the same time, I think it took me a lot longer to figure things out than it would have if I’d worked under a mentor. That would have been great, too -- there are pros and cons to both.\"", "6) It's no fun to play favorites. \"My favorite dish is always the next one I'm working on,\" Patterson says."
# ANSWER: "Daniel Patterson", "1) Eat your veggies.", "2) Enjoy said veggies a few weeks after their season starts.", "3) Ingredients dictate everything.", "4) Don't serve complex foods to your 4-year-old.", "5) You can succeed without a mentor."


# CLICKBAIT TEXT: "Human remains found in the search for missing teen"
# LINKED TEXT: "Authorities have uncovered human remains during their search for a missing Virginia teen.", "Norfolk Police said the remains were found near a shed in the backyard of a home on Smiths Ferry Road in Southampton County early Thursday, WAVY reports.", "Authorities visited the home on a tip as part of their investigation, but they did not say the remains were definitely those of 18-year-old Anjelica 'AJ' Hadsell.", "The Office of the Chief Medical Examiner will run tests to identify the remains.", "The college student was home for spring break when she disappeared from her house in the Tarrallton area of Norfolk on March 2, according to WTKR.", "\"On behalf of the Hadsell family, we would like to first thank the Norfolk Police Department, search and rescue, and all involved parties, businesses, individuals and media who have helped with the search for Anjelica Hadsell,\" the teen's mom, Jennifer, said in a statement released to the media. \"It has not been confirmed with the coroners office that [the remains are] in fact AJ. We ask that the media and public continue to respect the family in this time of adversity. Your thoughts and prayers for the Hadsell family are needed and appreciated.\"", "Previously, Jennifer Hadsell yelled at the media to \"please respect my family,\" when cameras showed up at her house, as seen in the clip below.", "In late March, Hadsell's father Wesley was arrested after he allegedly broke into the home of a person he believes kidnapped his daughter. Police did not release the name of the owners of the residence and have kept details of the case under wraps."
# ANSWER: "Southampton County", "Anjelica 'AJ' Hadsell"

#1: Goodbye Sapphire?
#2: New Home Button, New Limitations


In [ ]:
params = set_open_params()

response = get_chat_completion(prompt)

In [ ]:
IPython.display.Markdown(response.choices[0].message.content)

1) Goodbye Sapphire?
2) New Home Button, New Limitations

In [ ]:
from openai.error import InvalidRequestError
params = set_open_params()

index_list = []
response_list = []
gt_list = []

for row in val_data.iterrows():
  index = row[0]
  gt_postText = row[1]['postText']
  gt_targetParagraphs = row[1]['targetParagraphs']
  gt_spoiler = row[1]['spoiler']

  answer_postText = gt_postText[0]
  answer_targetParagraphs = ' '.join(gt_targetParagraphs)
  answer_spoiler = '\n'.join(gt_spoiler)

  prompt = """
      You are given a clickbait text and the corresponding linked large body of text. Extract the most important phrases from the linked text that can sarcastically answer the clickbait text.
      *Your answer must strictly be a list with more than one phrases. The phrases should strictly be brief (one word or few words).* Follow the two examples given below and answer the next.

      CLICKBAIT TEXT: {}
      LINKED TEXT: {}
      ANSWER: {}

      CLICKBAIT TEXT: {}
      LINKED TEXT: {}
      ANSWER:
      """.format(prompt_postText_1, prompt_targetParagraphs_1, prompt_spoiler_1, answer_postText, answer_targetParagraphs)
  
  try:
    response = get_chat_completion(prompt)

    index_list.append(index)
    response_list.append(response.choices[0].message.content)
    gt_list.append(answer_spoiler)
  except InvalidRequestError:
    print('============= Tokens greater that 4000 ==============')
    index_list.append(index)
    response_list.append('N/A')
    gt_list.append(answer_spoiler)

1
3
10
13
14
26
34
37
40
46
============= Tokens greater that 4000 ==============
47
62
64
70
78
84
90
91
93
108
110
111
113
116
117
118
123
124
125
============= Tokens greater that 4000 ==============
128
144
155
156
159
161
170
176
177
178
181
186
188
190
198
200
203
209
212
221
226
231
232
240
247
249
251
260
261
263
264
============= Tokens greater that 4000 ==============
267
270
285
290
296
303
305
314
335
336
337
342
351
353
356
358
366
379
380
393
394
396
398
399
406
414
418
429
451
452
453
457
470
472
473
============= Tokens greater that 4000 ==============
477
491
493
500
501
510
519
523
537
538
542
550
553
556
562
568
586
588
601
602
614
633
647
659
671
679
681
683
684
============= Tokens greater that 4000 ==============
690
692
694
700
702
703
707
724
731
============= Tokens greater that 4000 ==============
734
738
753
762
765
785
788
============= Tokens greater that 4000 ==============
789
790
797


In [ ]:
df = pd.DataFrame({'index': index_list,
                   'prediction': response_list,
                   'ground_truth': gt_list})

df

,index,prediction,ground_truth
0,1,1) 19 times intentionally violated orders.\n2)...,"""intentionally""\ncould transform a court case ..."
1,3,1) CBGB movie \n2) Alan Rickman \n3) Rupert Gr...,Alan Rickman & Rupert Grint\nCBGB
2,10,1) Mentally prepare.\n2) Do extra strength wor...,"Elettra Wiedemann\nextra strength work, so wei..."
3,13,1) Does she think Simpson really did it?\n2) W...,Marcia Clark\nDoes she think Simpson really di...
4,14,None needed. This is not really clickbait and ...,"Mitzi Gaynor\nBeverly Hills, California"
...,...,...,...
138,785,1) Reject isolation of modern society\n2) Deep...,BOOM – Portugal\nMeadows in the Mountains – Bu...
139,788,N/A,earned much of their fortune making unintentio...
140,789,"1) Klefki the key ring.\n2) Rotom, the pointle...",Klefki\nRotom\nLuvdisk\nUnknown (Unown)\nVanil...
141,790,1) Married eight times\n2) Insatiable appetite...,I don’t pretend to be an ordinary housewife\nP...


In [ ]:
df.to_csv('/content/drive/MyDrive/CS544/Project/webis-clickbait-22/out.csv', index=False)

## Evaluation

In [15]:
df = df.dropna(subset=['prediction'])
pred=list(df['prediction'])
predict_promt = []
for sentence in pred:
    cleaned_sentence = sentence.replace('\n', '').replace('\r', '').replace('\t', '').replace('\x0b', '').replace('\x0c', '')
    cleaned_sentence = re.sub(r'\d+\)|\n', '', cleaned_sentence)
    cleaned_sentence=cleaned_sentence.strip()
    predict_promt.append(cleaned_sentence)

print(predict_promt)


['19 times intentionally violated orders. Arpaio acknowledged violating the order. Arpaio said it was a mistake. Snow must decide if violations were intentional. Arpaio and aides failed to turn over video evidence. Arpaio hid thousands of pieces of evidence.', 'CBGB movie  Alan Rickman  Rupert Grint  Punk music evolution  Playing punk music.', 'Mentally prepare. Do extra strength work and planks for core. Cut out sugar for the week of the shoot.', 'Does she think Simpson really did it? Wants to be known as "Marcia Clark, the author of the Rachel Knight mystery series" and not "Marcia Clark, the O.J. Simpson prosecutor" Took up writing after the trial ended.', 'None needed. This is not really clickbait and does not require a sarcastic response.', 'Sponsored posts Fitness teas Hair removal #FitFam Waist training wraps Weight-loss products', 'Homemade oven cleaner with white vinegar and baking soda.  Soaking oven shelves in a resealable plastic bag with oven cleaner.  Using an old toothbr

In [16]:
ref=list(df['ground_truth'])
ref_promt = []
for sentence in ref:
    cleaned_sentence = sentence.strip().replace('\n', '').replace('\r', '').replace('\t', '').replace('\x0b', '').replace('\x0c', '') # remove leading/trailing whitespaces and newlines
    cleaned_sentence = re.sub(r'\d+\)|\n', '', cleaned_sentence)
    cleaned_sentence=cleaned_sentence.strip()
    ref_promt.append([cleaned_sentence])

print(ref_promt)


[['"intentionally"could transform a court case against Phoenix-area Sheriff Joe Arpaio from civil charges to a criminal prosecution'], ['Alan Rickman & Rupert GrintCBGB'], ['Elettra Wiedemannextra strength work, so weights, and quite a few planks for my core. My diet stayed pretty much the same, except I cut out sugar for the week of the shoot'], ['Marcia ClarkDoes she think Simpson really did it?'], ['Mitzi GaynorBeverly Hills, California'], ['Paige Hathaway – £3.8millionChantel Zales – £3.6 millionAna Cheri – £2.4 millionAbigail Ratchford – £2.3 millionClaudia Alende – £2.1 million'], ['homemade oven cleanerplace the shelves in a resealable plastic bag, spray with oven cleaner, seal the bag, then leave to soakold toothbrush is an essential oven-cleaning toolglass scraper is ideal for removing tough stainsremove greasy build-up on the hood of your oven with oil'], ['1. "You can\'t live your life according to maybes."2. "I wish there was a way to know you’re in the good old days before

In [17]:
# remove empty string elements from predictions and references
indices_to_remove = [i for i, p in enumerate(predict_promt) if p == ""]
for i in reversed(indices_to_remove):
    del predict_promt[i]
    del ref_promt[i]

print(predict_promt)  
print(ref_promt) 

['19 times intentionally violated orders. Arpaio acknowledged violating the order. Arpaio said it was a mistake. Snow must decide if violations were intentional. Arpaio and aides failed to turn over video evidence. Arpaio hid thousands of pieces of evidence.', 'CBGB movie  Alan Rickman  Rupert Grint  Punk music evolution  Playing punk music.', 'Mentally prepare. Do extra strength work and planks for core. Cut out sugar for the week of the shoot.', 'Does she think Simpson really did it? Wants to be known as "Marcia Clark, the author of the Rachel Knight mystery series" and not "Marcia Clark, the O.J. Simpson prosecutor" Took up writing after the trial ended.', 'None needed. This is not really clickbait and does not require a sarcastic response.', 'Sponsored posts Fitness teas Hair removal #FitFam Waist training wraps Weight-loss products', 'Homemade oven cleaner with white vinegar and baking soda.  Soaking oven shelves in a resealable plastic bag with oven cleaner.  Using an old toothbr

### BLEU Score

In [29]:

weights = (.01, 0,0,0)  # set weights for 4-grams
smoothing_func = SmoothingFunction().method1  # use smoothing method 1

total_bleu_score = 0
for i in range(len(predict_promt)):
    pred = predict_promt[i].split()
    refs = [ref.split() for ref in ref_promt[i]]
    bleu_score = sentence_bleu(refs, pred, weights=weights, smoothing_function=smoothing_func)
    total_bleu_score += bleu_score

average_bleu_score = total_bleu_score / len(predict_promt)
print("BLEU score:", average_bleu_score)

BLEU score: 0.5753608055204091


### METEOR Score

In [21]:
meteor = load_metric("meteor")


<ipython-input-21-a91dd7fa38e0>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric("meteor")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [24]:
score = meteor.compute(predictions=predict_promt, references=ref_promt)
alpha=.55
weighted_score = (1 - alpha) * score['meteor'] + alpha * len(predict_promt) / len(ref_promt)

weighted_score

0.6338074101166933